# 뻐꾸기 해싱(Cuckoo Hashing)

뻐꾸기 해싱은 뻐꾸기가 다른 새의 둥지에 알을 낳고, 부화된 뻐꾸기 새끼가 다른 새의 알이나 새끼들을 둥지에서 밀어내는 습성을 모방한 해싱방법이다.

<img src="./img/cuckoo.jpg" width="500"/>

### 핵심 아이디어
2개의 해시함수와 각 함수에 대응되는 해시테이블을 이용해 충돌이 발생하면 그 곳에 있는 키를 쫒아내자.

뻐꾸기 해싱은 2개의 해시함수와 2개의 해시테이블을 가지고 동작하게 됩니다.

뻐꾸기 해싱은 삽입 도중에 싸이클이 발생하게 될 경우 삽입 과정이 종료되지 않는다. 이러한 경우 재해시를 수행해야 합니다. 뻐꾸기 해싱의 장점은 탐색과 삭제를 100프로 보장은 아니지만 높은 확률로 O(1) 시간을 보장합니다.

In [9]:
h = [[None, None] for x in range(10+1)]
print(h)

[[None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None]]


In [19]:
g = [[None, None]] * 11
print(g)

[[None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None]]


In [18]:
class CuckooHashing: 
    def __init__(self, size): # 생성자
        self.M = size
        self.h = [[None, None] for x in range(size+1)]  # h-테이블
        self.d = [[None, None] for x in range(size+1)]  # d-테이블
        

    def hash(self, key):
        return key % self.M      # h-함수
    
    def hash2(self, key):
        return (key*key % 17) *11 % self.M  # d-함수
    
    def put(self, key, data): # 삽입 연산
        i = self.hash(key)
        print('i=', i)
        if self.h[i][0] == key:  # 이미 key 존재하면
            self.h[i][1] = data  # 데이터만 갱신
            return
        else:
            j = self.hash2(key) 
            if self.d[j][0] == key:  # 이미 key 존재하면
                self.d[j][1] = data  # 데이터만 갱신
                return 
        table = 'h'
        while True:
            if table == 'h': 
                if self.h[i][0] == None: # 삽입 위치 발견
                    self.h[i][0] = key   # key를 h-테이블에 저장
                    self.h[i][1] = data  # 데이터 저장 
                    print('h-table:[',i,']에 저장', self.h[i])
                    return           
                else: # 쫒아내고 저장
                    old_key  = self.h[i][0]
                    old_data = self.h[i][1] 
                    self.h[i][0] = key   # key를 h-테이블에 저장
                    self.h[i][1] = data  # 데이터 저장
                    print('[', old_key, ' ,', old_data,']를 h[',i,']에서 쫒아내고 ', self.h[i],'를 h[',i,']에 저장', )
                    i = self.hash2(old_key)
                    key  = old_key
                    data = old_data
                    table = 'd' 
            else: 
                if self.d[i][0] == None: # 삽입 위치 발견
                    self.d[i][0] = key   # key를 d-테이블에 저장
                    self.d[i][1] = data  # 데이터 저장 
                    print('d-table:[', i,']에 저장', self.d[i])
                    return           
                else: # 쫒아내고 저장
                    old_key  = self.d[i][0]
                    old_data = self.d[i][1] 
                    self.d[i][0] = key   # key를 d-테이블에 저장
                    self.d[i][1] = data  # 데이터 저장
                    print('[', old_key, ' ,', old_data,']를 d[',i,']에서 쫒아내고 ', self.d[i],'를 d[',i,']에 저장', )
                    i = self.hash(old_key)
                    key  = old_key
                    data = old_data
                    table = 'h' 
                
    def get(self, key): # 탐색 연산
        i = self.hash(key)
        if self.h[i][0] == key:
            return self.h[i][1] # 탐색 성공
        else:
            j = self.hash2(key) 
            if self.d[j][0] == key:
                return self.d[j][1] # 탐색 성공
        return None # 탐색 실패 
    
    def delete(self, key): # 삭제 연산
        i = self.hash(key)
        j = self.hash2(key)
        if self.h[i][0] == key:
            self.h[i][0] = None
            self.h[i][1] = None
            return 
        elif self.d[j][0] == key:
            self.d[i][0] = None
            self.d[i][1] = None
            return
        else:
            print('삭제하려는 키', key, '가 테이블에 없음')
            return
    
    def print_table(self):
        print('h-테이블:')
        for i in range(self.M):
            print('{:4}'.format(str(i)), ' ', end='')
        print()
        for i in range(self.M):
            print('{:4}'.format(str(self.h[i][0])), ' ', end='')
        print('\nd-테이블:')
        for i in range(self.M):
            print('{:4}'.format(str(i)), ' ', end='')
        print()
        for i in range(self.M):
            print('{:4}'.format(str(self.d[i][0])), ' ', end='')
        print()

if __name__ == '__main__':
    t = CuckooHashing(13)
    t.put(25, 'grape')      # 25:  12,   0
    t.put(43, 'apple')      # 43:   4,   0
    t.put(13, 'banana')     # 13:   0,   7
    t.put(26, 'cherry')     # 26:   0,   0
    t.put(39, 'mango')      # 39:   0,  10
    t.put(71, 'lime')       # 71:   9,   8
    t.put(50, 'orange')     # 50:  11,  11
    t.put(64, 'watermelon') # 64:  12,   7
    print('탐색 결과:')
    print('50의 data = ', t.get(50))
    print('64의 data = ', t.get(64))
    t.print_table() 
    print('-----  50  삭제 후:---------------')
    t.delete(50)
    t.print_table() 
    print('64의 data = ', t.get(64))
    print('-----  91  삽입 후:---------------')
    t.put(91, 'berry')
    t.print_table()
    print('--------------------------------')
    t.put(91, 'kiwi')       # 91:  0,   9
    t.print_table()                    

i= 12
h-table:[ 12 ]에 저장 [25, 'grape']
i= 4
h-table:[ 4 ]에 저장 [43, 'apple']
i= 0
h-table:[ 0 ]에 저장 [13, 'banana']
i= 0
[ 13  , banana ]를 h[ 0 ]에서 쫒아내고  [26, 'cherry'] 를 h[ 0 ]에 저장
d-table:[ 7 ]에 저장 [13, 'banana']
i= 0
[ 26  , cherry ]를 h[ 0 ]에서 쫒아내고  [39, 'mango'] 를 h[ 0 ]에 저장
d-table:[ 0 ]에 저장 [26, 'cherry']
i= 6
h-table:[ 6 ]에 저장 [71, 'lime']
i= 11
h-table:[ 11 ]에 저장 [50, 'orange']
i= 12
[ 25  , grape ]를 h[ 12 ]에서 쫒아내고  [64, 'watermelon'] 를 h[ 12 ]에 저장
[ 26  , cherry ]를 d[ 0 ]에서 쫒아내고  [25, 'grape'] 를 d[ 0 ]에 저장
[ 39  , mango ]를 h[ 0 ]에서 쫒아내고  [26, 'cherry'] 를 h[ 0 ]에 저장
d-table:[ 10 ]에 저장 [39, 'mango']
탐색 결과:
50의 data =  orange
64의 data =  watermelon
h-테이블:
0     1     2     3     4     5     6     7     8     9     10    11    12    
26    None  None  None  43    None  71    None  None  None  None  50    64    
d-테이블:
0     1     2     3     4     5     6     7     8     9     10    11    12    
25    None  None  None  None  None  None  13    None  None  39    None  None  
-----  50